# **TPS Jan 22 simple LSTM**

I create this notebook for the purpose of learning LSTM.

Referring to the following great notebooks.Thanks for sharing the information.

https://www.kaggle.com/thebrownviking20/intro-to-recurrent-neural-networks-lstm-gru

https://www.kaggle.com/samuelcortinhas/tps-jan-22-eda-modelling/notebook

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from itertools import combinations
import statistics
import time
from datetime import datetime

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, GRU, Bidirectional
from tensorflow.keras.optimizers import SGD
import math
from sklearn.metrics import mean_squared_error

In [ ]:
train_data = pd.read_csv('../input/tabular-playground-series-jan-2022/train.csv')
test_data = pd.read_csv('../input/tabular-playground-series-jan-2022/test.csv')

In [ ]:
y=train_data.num_sold
X=train_data.drop('num_sold', axis=1)

X.date=pd.to_datetime(X.date)
test_data.date=pd.to_datetime(test_data.date)

In [ ]:
def unofficial_hol(hol_path, df):
    countries = {'Finland': 1, 'Norway': 2, 'Sweden': 3}
    stores = {'KaggleMart': 1, 'KaggleRama': 2}
    products = {'Kaggle Mug': 1,'Kaggle Hat': 2, 'Kaggle Sticker': 3}
    
    # load holiday info.
    holiday = pd.read_csv(hol_path)
    
    fin_holiday = holiday.loc[holiday.country == 'Finland']
    swe_holiday = holiday.loc[holiday.country == 'Sweden']
    nor_holiday = holiday.loc[holiday.country == 'Norway']
    df['fin holiday'] = df.date.isin(fin_holiday.date).astype(int)
    df['swe holiday'] = df.date.isin(swe_holiday.date).astype(int)
    df['nor holiday'] = df.date.isin(nor_holiday.date).astype(int)
    df['holiday'] = np.zeros(df.shape[0]).astype(int)
    df.loc[df.country == 'Finland', 'holiday'] = df.loc[df.country == 'Finland', 'fin holiday']
    df.loc[df.country == 'Sweden', 'holiday'] = df.loc[df.country == 'Sweden', 'swe holiday']
    df.loc[df.country == 'Norway', 'holiday'] = df.loc[df.country == 'Norway', 'nor holiday']
    df.drop(['fin holiday', 'swe holiday', 'nor holiday'], axis=1, inplace=True)
    
    return df

In [ ]:
def date_feat_eng(df):
    df['day_of_week']=df['date'].dt.dayofweek       # 0 to 6
    df['day_of_month']=df['date'].dt.day            # 1 to 31
    df['weekend']=(df['day_of_week']//5 == 1)       # 0 or 1
    df['weekend']=df['weekend'].astype('int')       # int64
    df['week']=df['date'].dt.isocalendar().week     # 1 to 53
    df['week'][df['week']>52]=52                    # 1 to 52
    df['week']=df['week'].astype('int')             # int64
    df['month']=df['date'].dt.month                 # 1 to 12
    df['quarter']=df['date'].dt.quarter             # 1 to 4
    df['year']=df['date'].dt.year                   # 2015 to 2019
    df.drop('date',axis=1, inplace=True)            # drop date
    return df


In [ ]:
hol_path = '../input/public-and-unofficial-holidays-nor-fin-swe-201519/holidays.csv'

X = unofficial_hol(hol_path, X)
test_data = unofficial_hol(hol_path, test_data)

X= date_feat_eng(X)
test_data=date_feat_eng(test_data)

# Encoding
X=pd.get_dummies(X, columns=['store', 'country', 'product'])
test_data=pd.get_dummies(test_data, columns=['store', 'country', 'product'])

In [ ]:
X_train, y_train = np.array(X), np.array(y)

# Reshaping X_train for efficient modelling
X_train = np.reshape(X_train, (X_train.shape[0],X_train.shape[1],1))

I also tried the GRU architecture, but did not get good results.

In [ ]:
# The LSTM architecture

regressor = Sequential()
# First LSTM layer with Dropout regularisation
regressor.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1],1)))
regressor.add(Dropout(0.2))
# Second LSTM layer
regressor.add(LSTM(units=50, return_sequences=True))
regressor.add(Dropout(0.2))
# Third LSTM layer
regressor.add(LSTM(units=50, return_sequences=True))
regressor.add(Dropout(0.2))
# Fourth LSTM layer
regressor.add(LSTM(units=50))
regressor.add(Dropout(0.2))
# The output layer
regressor.add(Dense(units=1))

# Compiling the RNN
regressor.compile(optimizer='rmsprop',loss='mean_squared_error')
# Fitting to the training set
regressor.fit(X_train,y_train,epochs=50,batch_size=32)

In [ ]:
X_test = np.array(test_data)
X_test = np.reshape(X_test, (X_test.shape[0],X_test.shape[1],1))
predicted_value = regressor.predict(X_test)

In [ ]:
rows = X_test[:,0]
output = np.concatenate([rows, predicted_value],axis=1)

output = pd.DataFrame({'row_id': test_data.row_id,
                       'num_sold': np.squeeze(predicted_value)})

output.to_csv('submission.csv', index=False)